In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import pickle as pk
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Masking, Dense, LSTM, Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold
import evaluationutility

Using TensorFlow backend.


In [ ]:
#Trained LSTM

FOLDS = 5
MAX_POWER = 9
MAX_LAYER = 5
TRIALS = 5

gkf = GroupKFold(n_splits=FOLDS)
mms = MinMaxScaler()
ss = StandardScaler()

for source in ['raw', 'expert']:
    
    input_data = pk.load(open(f'data/padded_{source}_input.pkl', 'rb'))
    input_data = np.stack(input_data)
    target_data = pk.load(open(f'data/padded_{source}_target.pkl', 'rb'))
    target_data = np.stack(target_data)
    weight_data = pk.load(open(f'data/padded_{source}_weight.pkl', 'rb'))
    weight_data = np.equal(np.stack(weight_data), 1)
    group_data = np.array(pk.load(open(f'data/padded_{source}_sid.pkl', 'rb')))
    
    layers = []
    dimention = []
    auc = []
    kappa = []
    
    for lay in range(MAX_LAYER):
        for dim in np.power(2, np.arange(MAX_POWER+1)):
            print(lay, dim)
            
            aucs = []
            kappas = []
            
            for trials in range(TRIALS):
                for train_index, test_index in gkf.split(input_data, target_data, group_data):
                    
                    X_train, X_test = input_data[train_index,:,:], input_data[test_index,:,:]
                    y_train, y_test = target_data[train_index,:,:], target_data[test_index,:,:]
                    w_train, w_test = weight_data[train_index,:], weight_data[test_index,:]

                    keras.backend.clear_session()
                    lstm = Sequential()
                    lstm.add(Masking(mask_value=0.0, input_shape=(input_data[0].shape)))
                    for i in range(lay):
                        lstm.add(LSTM(dim, activation='tanh', return_sequences=True))
                    lstm.add(TimeDistributed(Dense(target_data.shape[2], activation='softmax')))
                    lstm.compile(optimizer='adam', loss='categorical_crossentropy', sample_weight_mode='temporal')

                    es = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0, restore_best_weights=True)]
                    lstm.fit(X_train, y_train, epochs=1000, validation_split=0.25, callbacks=es, verbose=False)

                    y_pred = lstm.predict(X_test)
                    test_p = []
                    test_t = []
                    for p, t, w in zip(y_pred, y_test, w_test):
                        test_p.append(p[w.flatten(), :])
                        test_t.append(t[w.flatten(), :])
                    test_p = np.concatenate(test_p)
                    test_t = np.concatenate(test_t)
                    test_p = mms.fit_transform(test_p)
                    aucs.append(evaluationutility.auc(test_t, test_p))
                    kappas.append(evaluationutility.cohen_kappa_multiclass(test_t, test_p))
                    
            layers.append(lay)
            dimention.append(dim)
            auc.append(np.mean(aucs))
            kappa.append(np.mean(kappas))
            print(f'auc: {auc[-1]}, kappa: {kappa[-1]}')
            
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], auc[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i} Hidden Layer LSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('ROC AUC')
    plt.title('Trained LSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()
    plt.figure()
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], kappa[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i} Hidden Layer LSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('Cohen\'s Kappa')
    plt.title('Trained LSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()

0 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

auc: 0.5534188576995415, kappa: 0.06843059645027565
0 2


In [ ]:
#Trained BiLSTM

FOLDS = 5
MAX_POWER = 9
MAX_LAYER = 5
TRIALS = 5

gkf = GroupKFold(n_splits=FOLDS)
mms = MinMaxScaler()
ss = StandardScaler()

for source in ['raw', 'expert']:
    
    input_data = pk.load(open(f'data/padded_{source}_input.pkl', 'rb'))
    input_data = np.stack(input_data)
    target_data = pk.load(open(f'data/padded_{source}_target.pkl', 'rb'))
    target_data = np.stack(target_data)
    weight_data = pk.load(open(f'data/padded_{source}_weight.pkl', 'rb'))
    weight_data = np.equal(np.stack(weight_data), 1)
    group_data = np.array(pk.load(open(f'data/padded_{source}_sid.pkl', 'rb')))
    
    layers = []
    dimention = []
    auc = []
    kappa = []
    
    for lay in range(MAX_LAYER):
        for dim in np.power(2, np.arange(MAX_POWER+1)):
            print(lay, dim)
            
            aucs = []
            kappas = []
            
            for trials in range(TRIALS):
                for train_index, test_index in gkf.split(input_data, target_data, group_data):
                    
                    X_train, X_test = input_data[train_index,:,:], input_data[test_index,:,:]
                    y_train, y_test = target_data[train_index,:,:], target_data[test_index,:,:]
                    w_train, w_test = weight_data[train_index,:], weight_data[test_index,:]

                    keras.backend.clear_session()
                    lstm = Sequential()
                    lstm.add(Masking(mask_value=0.0, input_shape=(input_data[0].shape)))
                    for i in range(lay):
                        lstm.add(LSTM(dim, activation='tanh', return_sequences=True))
                    lstm.add(LSTM(target_data.shape[2], activation='softmax', return_sequences=True))
                    lstm.compile(optimizer='adagrad', loss='categorical_crossentropy', sample_weight_mode='temporal')

                    es = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0, restore_best_weights=True)]
                    lstm.fit(X_train, y_train, epochs=1000, validation_split=0.25, callbacks=es, verbose=True)

                    y_pred = lstm.predict(X_test)
                    test_p = []
                    test_t = []
                    for p, t, w in zip(y_pred, y_test, w_test):
                        test_p.append(p[w, :])
                        test_t.append(t[w, :])
                    test_p = np.concatenate(test_p)
                    test_t = np.concatenate(test_t)
                    test_p = mms.fit_transform(test_p)
                    aucs.append(evaluationutility.auc(test_t, test_p))
                    kappas.append(evaluationutility.cohen_kappa_multiclass(test_t, test_p))
                    print(f'auc: {aucs[-1]}, kappa: {kappas[-1]}')
                    
            layers.append(lay)
            dimention.append(dim)
            auc.append(np.mean(aucs))
            kappa.append(np.mean(kappas))
            
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], auc[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i} Hidden Layer LSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('ROC AUC')
    plt.title('Trained BiLSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()
    plt.figure()
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], kappa[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i} Hidden Layer LSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('Cohen\'s Kappa')
    plt.title('Trained BiLSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()

In [ ]:
#Trained BiLSTM

FOLDS = 5
MAX_POWER = 9
MAX_LAYER = 4
TRIALS = 5

kf = KFold(n_splits=FOLDS, shuffle=True)
mms = MinMaxScaler()
ss = StandardScaler()

for source in ['raw', 'expert']:
    
    input_data = pk.load(open(f'data/padded_{source}_input.pkl', 'rb'))
    input_data = np.stack(input_data)
    target_data = pk.load(open(f'data/padded_{source}_target.pkl', 'rb'))
    target_data = np.stack(target_data)
    weight_data = pk.load(open(f'data/padded_{source}_weight.pkl', 'rb'))
    weight_data = np.equal(np.stack(weight_data), 1)
    
    layers = []
    dimention = []
    auc = []
    kappa = []
    
    for lay in range(1, MAX_LAYER+1):
        for dim in np.power(2, np.arange(MAX_POWER+1)):
            print(lay, dim)
            
            aucs = []
            kappas = []
            
            for trials in range(TRIALS):
                for train_index, test_index in kf.split(weight_data):
                    
                    X_train, X_test = input_data[train_index,:,:], input_data[test_index,:,:]
                    y_train, y_test = target_data[train_index,:,:], target_data[test_index,:,:]
                    w_train, w_test = weight_data[train_index,:], weight_data[test_index,:]

                    keras.backend.clear_session()
                    bilstm = Sequential()
                    bilstm.add(Bidirectional(LSTM(dim, activation='tanh', return_sequences=True), input_shape=(input_data[0].shape)))
                    for i in range(lay-1):
                        bilstm.add(Bidirectional(LSTM(dim, activation='tanh', return_sequences=True)))
                    bilstm.add(TimeDistributed(Dense(target_data.shape[2], activation='softmax')))
                    bilstm.compile(optimizer='adam', loss='categorical_crossentropy', sample_weight_mode='temporal')

                    es = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0, restore_best_weights=True)]
                    bilstm.fit(X_train, y_train, sample_weight=w_train, epochs=1000, validation_split=0.25, callbacks=es, verbose=True)

                    y_pred = bilstm.predict(X_test)
                    test_p = []
                    test_t = []
                    for p, t, w in zip(y_pred, y_test, weight_data):
                        test_p.append(p[w, :])
                        test_t.append(t[w, :])
                    test_p = np.concatenate(test_p)
                    test_t = np.concatenate(test_t)
                    aucs.append(evaluationutility.auc(test_t, test_p))
                    kappas.append(evaluationutility.cohen_kappa_multiclass(test_t, test_p))
                    print(f'auc: {aucs[-1]}, kappa: {kappas[-1]}')
                    
            layers.append(lay)
            dimention.append(dim)
            auc.append(np.mean(aucs))
            kappa.append(np.mean(kappas))
            
    plt.figure()
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], auc[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i+1} Layer BiLSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('ROC AUC')
    plt.title('Trained BiLSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()
    plt.figure()
    for i in range(MAX_LAYER):
        plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], kappa[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i+1} Layer BiLSTM')
    plt.xlabel('Hidden Layer Nodes')
    plt.ylabel('Cohen\'s Kappa')
    plt.title('Trained BiLSTM Results on ' + ('Raw' if source == 'raw' else 'Expert') + ' Features')
    plt.legend()
    plt.show()